<a href="https://colab.research.google.com/github/Amrod92/machine_learning/blob/master/Zalando_image_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from __future__ import absolute_import, division, print_function, unicode_literals

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

#print the tensorflow version
print(tf.__version__)

1.15.0-rc3


In [6]:
#importing Zalando fashion_mnist dataset
fashion_mnist = keras.datasets.fashion_mnist

#loading the dataset and returns an NumPy array of train_images and train_labels. It later tested against the test set test_images and test_labels arrays
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [0]:
##since the classification is made by number 0 to 9 we need to label these numbers to name
#0	T-shirt/top
#1	Trouser
#2	Pullover
#3	Dress
#4	Coat
#5	Sandal
#6	Shirt
#7	Sneaker
#8	Bag
#9	Ankle boot

class_names = ['T-Shirt/Top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle Boot']

In [11]:
#we are now going to explore the data
#the next line would tell the following (how_many_images, how_many_pixel, how_many_pixel)
print("this is how many images containes the database, and each image is rappresented as 28x28 pixels: ")
print (train_images.shape)
#the following line return how many images has been labeled...should return 60000
print("this would tell how many images has been labeled: ")
len(train_labels)

this is how many images containes the database, and each image is rappresented as 28x28 pixels: 
(60000, 28, 28)
this would tell how many images has been labeled: 


60000